In [1]:
#https://github.com/spro/practical-pytorch/blob/master/char-rnn-generation/char-rnn-generation.ipynb

import unidecode
import string
import random
import re

all_characters = string.printable
n_characters = len(all_characters)

file = unidecode.unidecode(open('data/warandpeace.txt').read())
file_len = len(file)
print('file_len =', file_len)

file_len = 3328586


In [2]:
chunk_len = 200

def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk())

particularly brotherly tenderness, 
almost lover-like. 

Owing to the count's customary carelessness 
nothing was ready for their departure by the 
twenty-eighth of August and the carts that 
were to c


In [3]:
import torch
import torch.nn as nn
from torch.autograd import Variable

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        input = self.encoder(input.view(1, -1))
        output, hidden = self.gru(input.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        return output, hidden

    def init_hidden(self):
        return torch.zeros(self.n_layers, 1, self.hidden_size)

In [4]:
# Turn string into list of longs
def char_tensor(string):
    tensor = torch.zeros(len(string)).long().unsqueeze(1)
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return tensor

print(char_tensor('abcDEF'))

tensor([[ 10],
        [ 11],
        [ 12],
        [ 39],
        [ 40],
        [ 41]])


In [5]:
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

In [6]:
def evaluate(prime_str='A', predict_len=100, temperature=0.8):
    hidden = decoder.init_hidden()
    prime_input = char_tensor(prime_str)
    predicted = prime_str

    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = decoder(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = decoder(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        
        # Add predicted character to string and use as next input
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = char_tensor(predicted_char)

    return predicted

In [7]:
import time, math

def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [8]:
def train(inp, target):
    hidden = decoder.init_hidden()
    decoder.zero_grad()
    loss = 0

    for c in range(chunk_len):
        output, hidden = decoder(inp[c], hidden)
        loss += criterion(output, target[c])

    loss.backward()
    decoder_optimizer.step()

    return loss.item() / chunk_len

In [ ]:
#n_epochs = 2000
n_epochs = 2000
print_every = 100
plot_every = 10
hidden_size = 100
n_layers = 1
lr = 0.005

decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

start = time.time()
all_losses = [0]
loss_avg = 0

for epoch in range(1, n_epochs + 1):
    #loss = train(*random_training_set())  
    loss = train(*random_training_set())
    loss_avg += loss

    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
        print(evaluate('Le', 500), '\n')


 16s (100 5%) 2.2805]
Lely efisrome tens 
to sos of ir wor he tho not soburs the contech mats sto sof the tha whe 
sons decende slol tyiter whe abss the stamed seaplones wafisssk sof ihd he 
aded atters oy wald an 
onf the to 
the 
hacke son the ferti and tsion he 
W. 
Sowas lle of ifler ar the 
sas to cof berer the Ro saslapblrstet tuptthe mof tulit the 
pred, haviche eshier lof 
berers sultise 
Id wamding Ink ent sente the 
nerting blomjsee inth- 
he watresse corich nom he at 
solve wabi hamt't lit oping 



wat hat mist thos to hime te the hre 
rere rermeplise it wemoke 
artrevtil tho fionthks bar, that at the coll lasl bltilc the 
"uly mof lencherse tso fiy the har, te thof laingitdes the Raf molfesser on the wofreme- fi 
onis of the mof wre lile he "roysser the thist, beve . the weper, pserrce bovale entard mlofs fener sites 
efing he- 
sust to lerey thaker that liblaser the serbe 
tiend sher thingels thy 

ofe cof tho thittor thalit hur vovingus nesy 
at merto wat soshd- 

ithe w